# Project 2 - Marcos Hernandez

Run the code in Ch6 regarding Polynomial Regression (pg120- 134 on BB)

Deliverables:
1. Submit graphs 6.4, 6.5, 6.6, 6.7, and 6.8.
2. Show the effect on accuracy when changing train/test ratio. <br>(find the statement used for that and change the ratio) submit the statement and accuracy for three different values (50-50, 75-25, and 85-15)
3. Submit a one-minute video of code explanation and the different outputs for the different train/test splits.

#Polynomial Regression

#Features ->Independent Variables(Sometimes also called explanatory varianbles)
#Label ->Dependent VariablE(The target to be predicted)

#Types of regression: Multiple regression, Polynomial Regression, Polynomial Multiple Regression

1. Simple Linear Regression-> a Linear relationship bewtween *one independent variable* and *one dependent variable*.
2. Multiple Regression-> Linear relationships between *two or more independent variables* and *one dependent variable*.
3. Polynomial Regression-> Modeling the relationships between *one independent variables* and *one dependent variable* using an *n^th degree polynomial function*.
4. Polynomial Multiple Regression-> Modeling the relationship between *two or more independent variables* and *one dependent variable* using an *n^th degree polynomial function*.

### Multiple Linear Regression 
- Using Boston housing and pricing dataset (found here: "http://lib.stat.cmu.edu/datasets/boston")





In [ ]:
import pandas as pd # we will need to import pandas to read the data and converted into a dataframe
import numpy as np # we will need to import numpy to convert the dataframe into a numpy array

data_url = "http://lib.stat.cmu.edu/datasets/boston" # the Boston dataset can be found in this url, since it has been removed from newer versions of sklearn
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None) # read the data from the url, the data starts at the 22nd row and has no header

#We first need to consolidate the data becase in the url, the last 3 columns spill over into the next row,
#Need to concatenate every other full column starting with the first column
#with the first two columns of every other column starting with the second column
# (excluding the third column because it is the target)
#this will give us the correct dataset and fix the spill over:
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]]) #[records(start-inclusive : end-exclusive : jupms),columns(start-inclusive : end-exclusive : jupms)]

#The target will simply be the third column in every other column starting at the second row 
# (this was the third column that spilled over but we excluded in the step above)
target = raw_df.values[1::2, 2] # the target [from record 1(using 0 indexing): to the end: jumping 2 so every other record]
#df=pd.DataFrame(data)
data #Printing the dataframe to validate the values, and format


array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [18]:
import matplotlib.pyplot as plt

print("The shape of the data is: ",data.shape)

#creating a dataframe, with the column names given in the dataset description
df=pd.DataFrame(data, columns=["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"] )

#Adding the target column to the dataframe
df["MEDV"]=target #this sets the data in the "MEDV",our target column, to the df column "MEDV" since it doesnt have one it creates it, and sets the values to the target

"""
BELOW ARE THE COLUMN NAMES AND DESCRIPTIONS OF THE BOSTON DATASET:
 CRIM     per capita crime rate by town
 ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
 INDUS    proportion of non-retail business acres per town
 CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
 NOX      nitric oxides concentration (parts per 10 million)
 RM       average number of rooms per dwelling
 AGE      proportion of owner-occupied units built prior to 1940
 DIS      weighted distances to five Boston employment centres
 RAD      index of accessibility to radial highways
 TAX      full-value property-tax rate per $10,000
 PTRATIO  pupil-teacher ratio by town
 B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
 LSTAT    % lower status of the population
 MEDV     Median value of owner-occupied homes in $1000's
 """
df.head()

The shape of the data is:  (506, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
#The boston dataset is a good dataset so no cleaning is necesary:
#this shows the datatypes and the columns in the df, as well as the number of non null values per column
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [ ]:
#To check if there is any missing values we use the following :
df.isnull().sum()
#this shows that all there are no missing values


CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

## Feature Selection

The next step is to select the features that are relevant to train the model. 
<br>The features that directly influence the result (which is prices of houses as shown below) to train the model. 

For this, we use the corr() function, which computes the pair wise correlation of columns:

In [23]:
corr= df.corr()
corr

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
CRIM,1.000000,-0.200469,0.406583,-0.055892,0.420972,-0.219247,0.352734,-0.379670,0.625505,0.582764,0.289946,-0.385064,0.455621,-0.388305
ZN,-0.200469,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995,0.360445
INDUS,0.406583,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800,-0.483725
CHAS,-0.055892,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929,0.175260
NOX,0.420972,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879,-0.427321
RM,-0.219247,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808,0.695360
AGE,0.352734,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339,-0.376955
DIS,-0.379670,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996,0.249929
RAD,0.625505,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676,-0.381626
TAX,0.582764,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993,-0.468536


A **positive correlatio** is a relationship between two variables that move **in tandem**. A positive correlation between variables exists when a variable decreases as the other variable decreses, and one variable increases as the other variable also increases. Similarly a negative correlation exists when one variable icreses and the other decreases.
<br> -1 represents a negative coorelation
<br>